In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
df_train=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
X_test=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")

In [3]:

X_train=df_train.drop("Class",inplace=False,axis=1)
Y_train=df_train["Class"]

transformer1 = ColumnTransformer(transformers=[
    ('remove_id_ej', 'drop', ['Id','EJ']),
    ('simple_impute',SimpleImputer(),['CB','CC','DU','FC','FL','FS','GL'])
    #('OHE',OneHotEncoder(drop='first'),['EJ'])

],remainder='passthrough')


transformer2 = ColumnTransformer(
    [('Knn_impute',KNNImputer(),slice(0, 55))]
    ,remainder='passthrough')


'''transformer3 = ColumnTransformer([
    ('pca', PCA(n_components=34),slice(0, 55)),
],remainder='passthrough')'''

Model = Pipeline([
    ('preprocessing', transformer1),
    ('impute', transformer2),
    #('pca', transformer3),
    ('rfc', RandomForestClassifier(random_state=0))
])

Model.fit(X_train,Y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('remove_id_ej', 'drop',
                                                  ['Id', 'EJ']),
                                                 ('simple_impute',
                                                  SimpleImputer(),
                                                  ['CB', 'CC', 'DU', 'FC', 'FL',
                                                   'FS', 'GL'])])),
                ('impute',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Knn_impute', KNNImputer(),
                                                  slice(0, 55, None))])),
                ('rfc', RandomForestClassifier(random_state=0))])

In [4]:
predictions=Model.predict_proba(X_test)
predictions

array([[0.52, 0.48],
       [0.52, 0.48],
       [0.52, 0.48],
       [0.52, 0.48],
       [0.52, 0.48]])

In [5]:
sub = X_test[['Id']].copy().copy()
sub['class_0'] = predictions[:,0]
sub['class_1'] = predictions[:,1]
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Id       5 non-null      object 
 1   class_0  5 non-null      float64
 2   class_1  5 non-null      float64
dtypes: float64(2), object(1)
memory usage: 248.0+ bytes


In [6]:
sub.to_csv('/kaggle/working/submission.csv', index=False)